In [0]:

#Mount ADLS Gen2
#Required each time the cluster is restarted which should be only on the first notebook as they run in order

#tiers = ["bronze", "silver", "gold"]
#adls_paths = [f"abfss://{tier}@rgdatabricksaccount.dfs.core.windows.net/" for tier in tiers]

# Accessing paths
#bronze_adls = adls_paths[0]
#silver_adls = adls_paths[1]
#gold_adls = adls_paths[2]

#dbutils.fs.ls(bronze_adls)
#dbutils.fs.ls(silver_adls)
#dbutils.fs.ls(gold_adls)

[]

In [0]:
from datetime import date, timedelta
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
# Retrieve the task value from the previous task (bronze)

bronze_output = dbutils.jobs.taskValues.get(taskKey="Bronze", key="bronze_output", debugValue={})

# Access individual variables
start_date = bronze_output.get("start_date", "")
bronze_adls = bronze_output.get("bronze_adls", "")
silver_adls = bronze_output.get("silver_adls", "")

print(f"Start Date: {start_date}, Bronze ADLS: {bronze_adls}")



Start Date: , Bronze ADLS: 


In [0]:

# Import necessarys libraries

from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df

DataFrame[geometry: struct<coordinates:array<double>,type:string>, id: string, properties: struct<alert:string,cdi:double,code:string,detail:string,dmin:double,felt:bigint,gap:double,ids:string,mag:double,magType:string,mmi:double,net:string,nst:bigint,place:string,rms:double,sig:bigint,sources:string,status:string,time:bigint,title:string,tsunami:bigint,type:string,types:string,tz:string,updated:bigint,url:string>, type: string]

In [0]:
df.head()


Row(geometry=Row(coordinates=[-147.2951, 64.9394, 0.0], type='Point'), id='ak025ae5hvt5', properties=Row(alert=None, cdi=None, code='025ae5hvt5', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak025ae5hvt5&format=geojson', dmin=None, felt=None, gap=None, ids=',ak025ae5hvt5,', mag=1.4, magType='ml', mmi=None, net='ak', nst=None, place='14 km WNW of Two Rivers, Alaska', rms=0.7, sig=30, sources=',ak,', status='automatic', time=1755215902817, title='M 1.4 - 14 km WNW of Two Rivers, Alaska', tsunami=0, type='earthquake', types=',origin,phase-data,', tz=None, updated=1755216036375, url='https://earthquake.usgs.gov/earthquakes/eventpage/ak025ae5hvt5'), type='Feature')

In [0]:
# Reshape earthquake data
df = (
    df
      .select(
    "id",
    col("geometry.coordinates").getItem(0).alias("longitude"),
    col("geometry.coordinates").getItem(1).alias("latitude"),
    col("geometry.coordinates").getItem(2).alias("elevation"),
    col("properties.title").alias("title"),
    col("properties.place").alias("place_description"),
    col("properties.sig").alias("sig"),
    col("properties.mag").alias("mag"),
    col("properties.magType").alias("magType"),
    col("properties.time").alias("time"),
    col("properties.updated").alias("updated")
)
)

In [0]:
df.head()


Row(id='ak025ae5hvt5', longitude=-147.2951, latitude=64.9394, elevation=0.0, title='M 1.4 - 14 km WNW of Two Rivers, Alaska', place_description='14 km WNW of Two Rivers, Alaska', sig=30, mag=1.4, magType='ml', time=1755215902817, updated=1755216036375)

In [0]:
# Validate data: Check for missing or null values
df = df \
    .withColumn("longitude", when(isnull(col("longitude")), 0).otherwise(col("longitude"))) \
    .withColumn("latitude", when(isnull(col("latitude")), 0).otherwise(col("latitude"))) \
    .withColumn("time", when(isnull(col("time")), 0).otherwise(col("time")))

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = df \
    .withColumn("time", (col("time") / 1000).cast(TimestampType())) \
    .withColumn("updated", (col("updated") / 1000).cast(TimestampType()))

In [0]:
df.head()


Row(id='ak025ae5hvt5', longitude=-147.2951, latitude=64.9394, elevation=0.0, title='M 1.4 - 14 km WNW of Two Rivers, Alaska', place_description='14 km WNW of Two Rivers, Alaska', sig=30, mag=1.4, magType='ml', time=datetime.datetime(2025, 8, 14, 23, 58, 22, 817000), updated=datetime.datetime(2025, 8, 15, 0, 0, 36, 375000))

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"



In [0]:
# Append DataFrame to Silver container in Parquet format
df.write.mode("append").parquet(silver_output_path)

In [0]:
dbutils.jobs.taskValues.set(key="silver_output", value=silver_output_path)